# Pixability Data Science Technical Assessment

The goal of this exerise is to create a *basic* machine learning model that uses the criteo dataset to predict the optimal *bid price* for the next period based on the features.

The criteo dataset is located in the `data.tsv` file. 

Instructions for the assignment, including a data dictionary for the Criteo dataset, can be found in the accompanying `Pixability_Data_Science_Assesment.pdf` file.

# Findings and Conclusions

An Ordinary Least Squares Linear Regression Model is a good optimal bid predictor for this data set, but not the best predictor. The model does not satisfy the Gauss-Markov Conditions for efficiency in a linear estimator. dodel adjusted $ R^2 $ was only 0.104 but $ RMSE $ was very good, only off on average by a fifth of a cent. The model errors will be very sensitive to inputs outside of a normal standard deviation.

    Train rmse: 0.0020056291132341726    
    Test rmse: 0.0020011741446683117   
    

# Methodology

The experiment was conducted as follows:

1. Load the dataset into memory
2. Explore the dataset
3. Clean and tidy the dataset
4. Select features to train the model.
5. Train the model and test for gauss markov conditions.  
6. Calculate model metrics ($ R^2, RMSE $)  

# Recommended Follow Up

1. Explore other non-linear models as they may provide better results. Models such as Random Forests, Decision-Trees, KNearestNeighbors, or a Deep Learning model will likely perform well.
2. Perform additional grid-searching or dimensionality reduction around feature selection.
3. Explore transforming the target variable to potentially address homosckedasticity issues. 
4. Build a model to forecast the seasonal/trend component of the cost to be used in an ensemble in making predictions.

## 1. Loading the DataSet

Because this is large data, we will only pull a sample of 1000 rows to explore some of the data values and make decisions as to how to clean/tidy the data.

First we install some packages.


In [ ]:
!pip install prince
!pip install sklearn
!pip install seaborn
!pip install matplotlib
!pip install statsmodels

In [ ]:
import pandas as pd

data_sample = pd.read_csv('data.tsv', sep='\t', nrows=1000)
data_sample.info()

## 2. Explore the Data

We can explore the sample data to get a look at some of the values, and some of the ranges of values.

In [ ]:
pd.set_option('display.max_columns', None)
data_sample.head()

Lets look at some of the possible value ranges:

In [ ]:
data_sample.describe()

## 3. Clean and Tidy the DataSet

Since the data is timeseries and sorted by timestamp, we cannot run any exploratory plots on the data just yet. Instead, let's import the data into chunks and process it so that we can eliminate columns that are not necessary and try to fit the full data in memory.  

It looks like `uid, conversion_timestamp, conversion_id, attribution` are not useful features to include because they are metadata about the user/conversion that are either known after the fact or should not impact the final decision.

We also need to change the different cat columns into categoricals/dummy variables so that they are not interpreted as numericals.

We assume that the cost for the ad is handled like most ad bidding systems where the advertiser only pays one penny more than the 2nd highest bidder, which means that all `cost` values for impressions resulting in conversions are "optimal". We will not consider `cpo` for this model as we do not have any information on order profitability so it cannot be used to impact our decision-making in terms of an optimal bid.

Now we will import the data by chunks, excluding those columns, dropping any record that did not result in a conversion and did not result in a positive cost and performing the necessary cleaning/tidying steps.

The file size is ~2GB, so it should roughly fit into available memory, especially after preprocessing.

In [ ]:
def preprocess(data):
    data.drop(['uid', 'conversion_timestamp', 'conversion_id', 'attribution', 'cpo'], axis='columns', inplace=True)
    data = data.loc[data['conversion'] == 1]
    return data

data_list = []

for chunk in pd.read_csv('data.tsv', sep='\t', chunksize=10000):
    preprocessed_chunk = preprocess(chunk)
    data_list.append(preprocessed_chunk)

data_tidied = pd.concat(data_list)
data_tidied[['campaign', 'cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9']] = data_tidied[['campaign', 'cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9']].astype('category')
data_tidied.index = data_tidied['timestamp']
data_tidied.drop(['conversion','timestamp','click'], axis='columns', inplace=True) # Conversion / Click always 1
data_tidied.info()

We were able to reduce the size of the dataset to 41.5 MB after initial preprocessing.

Now let's seperate the target variable from the dataset, and move on to do some feature selection.

In [ ]:
Y = data_tidied['cost']
X = data_tidied.drop('cost', axis='columns')
print(Y.head())
print(X.head())

## 4. Feature Selection

First let's separate the categorical features from the numeric features so we can work with them individually.


In [ ]:
X_categorical = X.loc[:, ['campaign', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']]
X_numeric = X.loc[:, ['click_pos', 'click_nb', 'time_since_last_click']]

print(X_categorical.info())
print(X_numeric.info())

Let's start by analyzing for multicolinearity in the features.

In [ ]:
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

sns.heatmap(X_numeric.corr())

This chart suggests that there is strong multicolinearity between click_pos and click_nb. One of them will have to be dropped.

In [ ]:
print(Y.corr(X_numeric['click_pos']))
Y.corr(X_numeric['click_nb'])


There is a slightly higher correlation between `click_nb` and the target than `click_pos` so let's drop `click_pos`. Other methods of deciding which variable to drop include dropping the column with the higher VIF, or training models and comparing $ R^2 $ values

In [ ]:
X_numeric_decolineated = X_numeric.drop(['click_pos'], axis='columns')
sns.heatmap(X_numeric_decolineated.corr())

In [ ]:
X_numeric_decolineated.info()

Because we have so many categorical values, let's use frequency encoding to reduce the dimensionality. Frequency encoding will weight each of the categories according to how often they show up in the dataset. This allows us to avoid the curse of dimensionality while possibly preserving some information about the recurrence of certain features associated with conversions.

In [ ]:
def frequency_encode(column, df):
    category = (X_categorical.groupby(column).size()) / len(X_categorical)
    df[f'{column}_encode'] = df[column].apply(lambda x: category.loc[x])
    return df

frequency_encode('cat1', X_categorical)
frequency_encode('cat2', X_categorical)
frequency_encode('cat3', X_categorical)
frequency_encode('cat4', X_categorical)
frequency_encode('cat5', X_categorical)
frequency_encode('cat6', X_categorical)
frequency_encode('cat7', X_categorical)
frequency_encode('cat8', X_categorical)
frequency_encode('cat9', X_categorical)
frequency_encode('campaign', X_categorical)

X_categorical_encoded = X_categorical.drop(['campaign', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9'], axis='columns')
X_categorical_encoded.info()

In [ ]:
sns.heatmap(X_categorical_encoded.corr())

In [ ]:
print(Y.corr(X_categorical_encoded['cat3_encode']))
print(Y.corr(X_categorical_encoded['campaign_encode']))

`cat3` has higher correlation with the target variable, and high multicolinearity with `campaign`, so we will drop `campaign`.

In [ ]:
X_categorical_decolineated = X_categorical_encoded.drop('campaign_encode', axis='columns')
sns.heatmap(X_categorical_decolineated.corr())

In [ ]:
X_full = pd.concat([X_numeric_decolineated, X_categorical_decolineated], axis='columns')
sns.heatmap(X_full.corr())

Let's select the top five features. We selected a k of ten arbitrarily here for this exercise. k can be optimized by comparing the results of models iteratively for as many k as there are feature columns, and counting down from columns-1, ... 1.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

selection = SelectKBest(f_regression, k=10).fit(X_full, Y)
features = X_full.columns[selection.get_support()].tolist()
print(features)
X_selected = X_full.loc[:, features]
X_selected.head()

## 5. Train the model and forecast

This step involves -   
1. Detecting and correcting for seasonality/trend.  
2. Splitting the data into a train/test set.  
3. Train the model and calculate errors.  
4. Determine if the model satisfies the Gauss Markov conditions.  


### Fast Fourier Transform

We detect the different periods of seasonality using a fast fourier transform (FFT). FFT works by decomposing the signal into it's component frequencies. This allows us to identify strong recurring frequencies to determine a period for seasonality.

In [ ]:
from scipy import stats, special, fft, signal as sig
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np

fft_results = fft.fft(np.array(Y))
power = np.abs(fft_results)
freq = fft.fftfreq(len(Y))
mask = freq >= 0
freq = freq[mask]
power = power[mask]

peaks = sig.find_peaks(power[freq >= 0])[0]
peak_freq = freq[peaks]
peak_power = power[peaks]
period = 1/peak_freq
fft_output = pd.DataFrame()

fft_output['index'] = peaks
fft_output['freq'] = peak_freq
fft_output['amplitude'] = peak_power
fft_output['period'] = period
fft_output['period_rounded'] = np.round(fft_output['period']).astype(int)
fft_output['fft'] = fft_results[peaks]
fft_output.sort_values('amplitude', ascending=False, inplace=True)

fft_output.head()

Fast Fourier transform detects a strong signal every 26006 periods within timestamp.

We can now use the seasonal_decompose functions to remove the seasonality.

In [ ]:
seasonal_output = seasonal_decompose(Y, period=int(fft_output.iloc[0]['period_rounded']))
seasonal_output.plot()

In [ ]:
trend = pd.DataFrame(seasonal_output.trend).fillna(0)
detrended_signal = pd.DataFrame(Y - trend['trend'])
detrended_signal.columns = ['detrended']
detrended_seasonality = seasonal_decompose(detrended_signal, period=int(fft_output.iloc[0]['period_rounded']))
decycled_signal = detrended_signal['detrended'] - detrended_seasonality.seasonal
decycled_signal = pd.DataFrame(decycled_signal)
decycled_signal.columns = ['decycled']
decycled_signal.head()

Let's test to see if the signal is thoroughly deseasonalized by leveraging the augmented dickey-fuller test (ADF). ADF tests for the presence of a unit root, which implies seasonality. The null hypothesis is that a unit root exists, and to reject the null means that the data is stationary and has been deseasonalized. We seek to validate stationarity at a 95% confidence level so we seek a p-value of 0.05 or less. We will use a subsample of the time series data to test for stationarity for performance reasons.

In [ ]:
from statsmodels.tsa.stattools import adfuller as adf

stat = adf(decycled_signal.loc[0:50000,'decycled'])
print(f"Statistic is {stat[0]}")
print(f"P-value is {stat[1]}")

The p-value is less than our alpha of 0.05, thus we reject the null hypothesis and conclude the data is likely stationary.

We can use the KPSS test to test for trend-stationarity in the same fashion, but we will skip that for this exercise.

With our decycled signal, we no longer need to worry about time series in our data and we can obtain a train/test split of our X and Y values so that we can train a linear regression model.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_selected)
a = 0.01 - decycled_signal.min()
X_scaled = scaler.transform(X_selected)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, decycled_signal, train_size=0.8, random_state=42)

We will use statsmodels to produce a Linear Regression model and evaluate metrics to determine how effective it is at making predictions.

In [ ]:
import statsmodels.api as sm
import statsmodels.stats.api as sms

#Don't forget to add the constant to the dataset with statsmodels package
X_train_constant = sm.add_constant(X_train)
X_test_constant = sm.add_constant(X_test)



model = sm.OLS(y_train, X_train_constant).fit()
model.summary()

The Jarque-Bera test for normality rejects the null hypothesis that the residuals of the model are normally distributed at a 95% confidence level.

Let's check the homoskedasticity of the errors:

In [ ]:
test = sms.het_breuschpagan(model.resid, model.model.exog)
stat, pval, fscore, fpval = test
print(f'Lagrange statistic: {stat}')
print(f'P-value: {pval}')
print(f'F-Score: {fscore}')
print(f'F-Score P-Value: {fpval}')


The Breusch-Pagan test rejects the null that the errors are homoskedastic at a 95% confidence level.

Based on the above, we can conclude that our linear model does not satisfy the Gauss-Markov conditions and is therefore not the best estimator. It is recommended that other estimators be explored to determine if greater precision from predictions can be achieved.

## 6. Calculate RMSE

In [ ]:
from statsmodels.tools.eval_measures import rmse

y_predicted_train = model.predict(X_train_constant)
residuals_train = (y_train['decycled'] - y_predicted_train)
residualavg_train = (residuals_train**2).sum()
rmse_train = np.sqrt(residualavg_train / len(residuals_train))
print(f'Train rmse: {rmse_train}')


y_predicted_test = model.predict(X_test_constant)
residuals_test = (y_test['decycled'] - y_predicted_test)
residualavg_test = (residuals_test**2).sum()
rmse_test = np.sqrt(residualavg_test / len(residuals_test))
print(f'Test rmse: {rmse_test}')

print(f"Y Train Min: {y_train['decycled'].min()}")
print(f"Y Train Max: {y_train['decycled'].max()}")